In [ ]:
import sys
import os
dir = os.path.abspath('')
while not dir.endswith('ardt'): dir = os.path.dirname(dir)
if not dir in sys.path: sys.path.append(dir)

In [ ]:
import numpy as np

from datasets import load_dataset, load_from_disk
from huggingface_hub import login

from utils.helpers import find_root_dir

from access_tokens import HF_WRITE_TOKEN

In [ ]:
login(token=HF_WRITE_TOKEN)

In [ ]:
ARDT_DIR = find_root_dir()

In [ ]:
TYPE = 'train'
VERSION = 'v2'

In [ ]:
dataset = load_from_disk(f'{ARDT_DIR}/datasets-raw/nonadv-raw-data/ppo_halfcheetah_v2_train_v2')

In [ ]:
total_max = 0
total_min = 0

for t in dataset:
    total_max = max(total_max, np.sum(t['rewards']))
    total_min = min(total_min, np.sum(t['rewards']))
    
print("Max total reward:", total_max)
print("Min total reward:", total_min)

In [ ]:
# run if we want to add a 0 adversary
pr_actions = dataset['actions']
new_pr_actions = []
adv_actions =[]
for tr in pr_actions:
    pr_l = []
    adv_l = []
    for a in tr:
        adv = np.array(a).astype(np.float32) * 0.0
        pr = np.array(a).astype(np.float32)
        pr_l.append(list(pr))
        adv_l.append(list(adv))
    new_pr_actions.append(pr_l)
    adv_actions.append(adv_l)

dataset = dataset.add_column('pr_actions', new_pr_actions)
dataset = dataset.add_column('adv_actions', adv_actions)
dataset = dataset.remove_columns(['actions'])

In [ ]:
dataset.save_to_disk(f'{ARDT_DIR}/datasets/ppo_noadv_{TYPE}_halfcheetah_{VERSION}')